<a href="https://colab.research.google.com/github/2003Yash/semantic-search/blob/main/semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

Import Libraries

In [ ]:
# sentence tranformers will take a sentence with full of words and convert them into a single context aware embedding
# Flowchart : sentence -> BERT -> Pooling -> Embeddings
# other methods like unigram, n-gram, avg of all vectors in a sentence to create a dense vector to represent
# sentence tranformer is superior than anything from abov methods as it can capture bi-directional contexts

In [2]:
from sentence_transformers import SentenceTransformer
import faiss # faiss is a high-performance library for similarity search, used here for fast retrieval.
import numpy as np

In [3]:
# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Input-data:

In [6]:
# List of world news headlines
news_headlines = [
    "Global economy recovers as inflation slows down",
    "New species of butterfly discovered in the Amazon",
    "Scientists develop breakthrough cancer treatment",
    "Wildfires continue to spread in Australia",
    "Stock markets crash amid geopolitical tensions",
    "Countries sign historic climate change agreement",
    "Major earthquake causes damage in Japan",
    "AI technology helps doctors diagnose diseases faster",
    "New educational reforms promise better schooling system",
    "Conflict in Middle East escalates further"
]

# Define a query representing "good news"
query = ["Positive global developments, scientific breakthroughs, environmental success stories"]

Embedding input data:

In [7]:
# Encode the news headlines and query into embeddings
news_embeddings = model.encode(news_headlines, normalize_embeddings=True)
query_embedding = model.encode(query, normalize_embeddings=True)

 Build a FAISS Index for Similarity Search

In [8]:
# Create a FAISS index for similarity search
# - Faiss is a library — developed by Facebook AI — that enables efficient similarity search.
#    So, given a set of vectors, we can index them using Faiss
# — then using another vector (the query vector), we search for the most similar vectors within the index.

In [ ]:
dimension = news_embeddings.shape[1]  # news-emdedding has many words and many dimensions for each word and .shape[1] returns the no.of features in the embedding.
                                      # ie., of news_articles has 10 sentences and each has 384 dims of emdedings then 10x384 is size and shape[1] is 384 returns no.od features for faiss to work on
index = faiss.IndexFlatIP(dimension)  # creates a FAISS index that performs similarity search using Inner Product (dot product) within dimensions. AKA cosine similarity

In [ ]:
index.add(news_embeddings) # adds all the encoded news headlines to the FAISS search index.

Semantic Search

In [9]:
k = 3 # k means we retrieve the top 3 most relevant news articles.
D, I = index.search(query_embedding, k) # index.search(query_embedding, k) returns:
                                        # D: Similarity scores (higher is better)
                                        # I: Indices of the top k most relevant headlines.

Print Output

In [10]:
# Print the top 3 good news headlines
print("Top Good News Articles:")
for idx in I[0]:
    print(f"- {news_headlines[idx]}")

Top Good News Articles:
- Scientists develop breakthrough cancer treatment
- Countries sign historic climate change agreement
- Global economy recovers as inflation slows down
